In [2]:
import cv2
import numpy as np

def read_img(filename):
    img = cv2.imread(filename)
    return img

def edge_detection(img, line_wdt, blur):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Corrected the typo
    grayBlur = cv2.medianBlur(gray, blur)
    edges = cv2.adaptiveThreshold(grayBlur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_wdt, blur)  # Corrected the typo
    return edges

def color_quantization(img, k):
    data = np.float32(img).reshape((-1, 3))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)
    ret, label, center = cv2.kmeans(data, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    result = center[label.flatten()]
    result = result.reshape(img.shape)
    return result

# Replace 'your_image.jpg' with the path to your input image
filename = '/content/2.jpeg'

# Read the input image
img = read_img(filename)

# Apply edge detection
line_wdt = 9
blur_value = 7
totalColors = 4
edgeImg = edge_detection(img, line_wdt, blur_value)

# Apply color quantization
cartoon = color_quantization(img, totalColors)

# Apply bilateral filter for a smoother result (optional)
blurred = cv2.bilateralFilter(img, d=7, sigmaColor=200, sigmaSpace=200)
cartoon = cv2.bitwise_and(blurred, blurred, mask=edgeImg)

# Save the cartoonized image
cv2.imwrite('res.jpg', cartoon)


True

In [3]:
import cv2
import numpy as np

# Load the input image
input_image = cv2.imread('/content/2.jpeg')

# Convert the image to grayscale
gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

# Apply median blur to reduce noise
gray = cv2.medianBlur(gray, 5)

# Detect edges using adaptive thresholding
edges = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 9)

# Apply bilateral filter to smooth the image while preserving edges
color = cv2.bilateralFilter(input_image, 9, 300, 300)

# Combine the edges and smoothed color image to create a cartoon effect
cartoon = cv2.bitwise_and(color, color, mask=edges)

# Save the cartoonized image
cv2.imwrite('cartoonized_image.jpg', cartoon)


True

In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import files
from IPython.display import Image, display

# Function to resize and crop the image
def resize_crop(image):
    h, w, c = np.shape(image)
    if min(h, w) > 720:
        if h > w:
            h, w = int(720 * h / w), 720
        else:
            h, w = 720, int(720 * w / h)
    image = cv2.resize(image, (w, h), interpolation=cv2.INTER_AREA)
    h, w = (h // 8) * 8, (w // 8) * 8
    image = image[:h, :w, :]
    return image

# Define the cartoonize function using TensorFlow Hub
def cartoonize(image):
    try:
        # Load the cartoonization model from TensorFlow Hub
        model_url = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'  # Pre-trained cartoonization model
        cartoonize_model = hub.load(model_url)

        image = resize_crop(image)

        # Generate cartoon-style image
        cartoon_image = cartoonize_model(tf.convert_to_tensor(image))[0]
        cartoon_image = (cartoon_image * 255).numpy().astype(np.uint8)

        return cartoon_image
    except Exception as e:
        return None

# Function to upload an image and display the cartoonized result
def upload_and_cartoonize():
    uploaded = files.upload()
    for filename in uploaded.keys():
        image = cv2.imread(filename)
        cartoon_image = cartoonize(image)
        if cartoon_image is not None:
            # Display the original and cartoonized images
            display(Image(filename=filename, width=400))
            display(Image(data=cv2.imencode('.jpg', cartoon_image)[1].tobytes(), width=400))

# Call the upload_and_cartoonize function to upload an image and see the result
upload_and_cartoonize()


Saving 3.jpg to 3.jpg


In [6]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model

# Load a single real image
input_image_path = '/content/1.jpeg'
target_size = (256, 256)
input_image = load_img(input_image_path, target_size=target_size)
input_image = img_to_array(input_image)
input_image = np.expand_dims(input_image, axis=0)

# Define the dimensions for the latent space
latent_dim = 128

# Encoder
input_layer = Input(shape=(256, 256, 3))
x = Flatten()(input_layer)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)

# Sampling from the latent space
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], latent_dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = tf.keras.layers.Lambda(sampling)([z_mean, z_log_var])

# Define the encoder model
encoder_model = Model(input_layer, [z_mean, z_log_var, z])

# Decoder
decoder_input = Input(shape=(latent_dim,))
x = Dense(256, activation='relu')(decoder_input)
x = Dense(512, activation='relu')(x)
x = Dense(256*256*3, activation='sigmoid')(x)
decoded_output = Reshape((256, 256, 3))(x)

# Define the decoder model
decoder_model = Model(decoder_input, decoded_output)

# Generate an avatar-style image from the latent representation
latent_representation = encoder_model.predict(input_image)
avatar_image = decoder_model.predict(latent_representation[2])

# Rescale the avatar image to the original size (if needed)
avatar_image = (avatar_image * 255).astype(np.uint8)

# Save the generated avatar
output_image_path = 'avatar_output.jpg'  # Replace with the desired output path
save_img(output_image_path, avatar_image[0])


1/1 [==============================] - 0s 138ms/step
